# Get All EC2 Instance with DisableApiTermination = False

Ensure that the **EC2 instances provisioned outside of the AWS Auto Scaling Groups (ASGs)** have Termination Protection safety feature enabled in order to protect your instances from being accidentally terminated.

In [ ]:
import os, sys
from datetime import datetime
import pandas as pd
import boto3
from IPython.display import display
from botocore.config import Config

# 0. Core Function

### Credential Part

In [ ]:
def createAWSCLISession(aws_access_key_id="", aws_secret_access_key="", aws_session_token="", credential_profile=""):
    if aws_access_key_id != "" and aws_secret_access_key != "":
        # Can Specific Key Here
        session = boto3.Session(
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            aws_session_token=aws_session_token
        )

    if credential_profile != "":
        session = boto3.Session(profile_name=credential_profile)
    
    return session

### EC2 Instances

In [ ]:
def extractEC2InstanceName(tags=[]):
    for tag in tags:
        if tag["Key"]=="Name":
            return tag['Value']

In [ ]:
def extactEC2Instance(reservation=[]):
    return [{"instanceId":i["InstanceId"], "instanceName":extractEC2InstanceName(i["Tags"]),"tags":i["Tags"]} for i in reservation["Instances"]]

In [ ]:
def getAllEC2InstanceIDByReservations():
    response = EC2.describe_instances()
    return { res["ReservationId"]:extactEC2Instance(reservation=res) for res in response["Reservations"]}

### Disable Api Termination Status

In [ ]:
def getDisableApiTerminationStatus(instanceID=""):
    response = EC2.describe_instance_attribute(Attribute='disableApiTermination', InstanceId=instanceID)    
    return response["DisableApiTermination"]["Value"]

In [ ]:
def getAndextractDisableApiTerminationStatus(credential_profile="", reservationId="", data={}):
    return {
        "cloudAccountName": credential_profile,
        "reservationId": reservationId,
        "assetID": data["instanceId"],
        "assetName": extractEC2InstanceName(data["tags"]),
        "disableApiTermination": getDisableApiTerminationStatus(data["instanceId"]),
        "tags": data["tags"]
    }

In [ ]:
def getDisableApiTerminationStatusFromReservation(reservationAssets={}):
    disableApiTerminationStatusAll = []
    for key, instances in ec2_reserved.items():
        print("Reservation ID : " + key)
        print("  Instance ID : ", end=" ")
        for i in instances:
            print(i["instanceId"], end=" ")
            disableApiTerminationStatusAll.append(getAndextractDisableApiTerminationStatus(credential_profile=credential_profile, reservationId=key, data=i))
        print()
    return disableApiTerminationStatusAll

# 1. Check EC2TerminationProtection of One Account

In [ ]:
credential_profile = "<Profile Name>"
region_name = 'ap-southeast-1'
EC2config = Config(
    region_name = region_name
)
credential_profile

In [ ]:
session = createAWSCLISession(credential_profile=credential_profile)
EC2 = session.client('ec2', config=EC2config)

In [ ]:
ec2_reserved = getAllEC2InstanceIDByReservations()

for k, v in ec2_reserved.items():
    print("Reservation ID : "  + str(k))
    display(pd.DataFrame(v))

In [ ]:
disableApiTerminationStatusAll = getDisableApiTerminationStatusFromReservation(reservationAssets=ec2_reserved)

In [ ]:
disableApiTerminationStatusAll_df = pd.DataFrame(disableApiTerminationStatusAll)
disableApiTerminationStatusAll_df

In [ ]:
display(disableApiTerminationStatusAll_df[disableApiTerminationStatusAll_df["disableApiTermination"]==True])

Filter EC2 only outside of the AWS Auto Scaling Groups

In [ ]:
display(disableApiTerminationStatusAll_df[((~disableApiTerminationStatusAll_df['assetName'].str.lower().str.contains('eks|batch'))) & (disableApiTerminationStatusAll_df["disableApiTermination"]==False)])

# 2. Check EC2TerminationProtection of Multiple Account

In [ ]:
credential_profiles = [
    "<Profile Name>",
]

In [ ]:
region_name = 'ap-southeast-1'
EC2config = Config(
    region_name = region_name
)

In [ ]:
def checkEC2TerminalProtectionInOneAccount(credential_profile=""):
    print("Cloud Account : " + credential_profile)
    
    session = createAWSCLISession(credential_profile=credential_profile)

    EC2 = session.client('ec2', config=EC2config)
    ec2_reserved = getAllEC2InstanceIDByReservations()
    
    disableApiTerminationStatusAll = getDisableApiTerminationStatusFromReservation(reservationAssets=ec2_reserved)
    return disableApiTerminationStatusAll

In [ ]:
disableApiTerminationStatusAllAccounts = []
for credential_profile in credential_profiles:
    print("\nCredential Profile : " + credential_profile)
    disableApiTerminationStatusAllAccounts += checkEC2TerminalProtectionInOneAccount(credential_profile=credential_profile)

In [ ]:
disableApiTerminationStatusAllAccounts_df = pd.DataFrame(disableApiTerminationStatusAllAccounts)
disableApiTerminationStatusAllAccounts_df

In [ ]:
result_df = pd.concat([disableApiTerminationStatusAllAccounts_df, disableApiTerminationStatusAllAccounts_df["tags"].apply(pd.Series)], axis=1)
result_df = result_df.drop(columns="tags")
len(result_df.index)

In [ ]:
display(result_df[result_df["disableApiTermination"]==True])

In [ ]:
display(result_df[((~result_df['assetName'].str.lower().str.contains('eks|batch'))) & (result_df["disableApiTermination"]==False)])

In [ ]:
# nowTime = datetime.now().strftime("(%y%m%d-%H%M%S) ")
# filepath="outputs/" + nowTime + "EC2 Termination Protection"

In [ ]:
# result_df.to_excel(filepath  + ".xlsx", sheet_name='disableApiTermination', index=False, encoding='utf-8')